In [1]:
# Supervised learning using a feedforward network

In [3]:
# Get and preprocess the data according to our requirement
import pandas as pd
data = pd.read_csv('insurance.csv')
print('\n #Features = '+str(data.shape[1]))
print('\n #Samples = '+str(data.shape[0]))
data.head()


 #Features = 2

 #Samples = 63


,Number of claims,Payment
0,108,392.5
1,19,46.2
2,13,15.7
3,124,422.2
4,40,119.4


In [80]:
# Following is the code for the training network.
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
from numpy import random

torch.manual_seed(47)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from torch.nn.functional import normalize

dat = data.to_numpy(dtype= np.float32) # dataframe  to numpy 
# Split data into input and labels
x = torch.from_numpy(dat[:,[0]])
x = torch.nn.functional.normalize(x, p=2.0, dim = 0)
y = torch.from_numpy(dat[:,[1]])

# Split data into test and train
train_size = int(0.8*len(dat))
test_size = len(dat)- train_size
x_train, x_test = torch.utils.data.random_split(x, [train_size, test_size])
y_train, y_test = torch.utils.data.random_split(y, [train_size, test_size])

# define train and test Datasets
class CustomDataset_train(Dataset):
    def __init__(self):
        self.x_train = x_train
        self.y_train = y_train
        self.num_samples = len(x_train)
        
    def __getitem__(self, index):
        return self.x_train[index], self.y_train[index]

    def __len__(self):
        return self.num_samples

class CustomDataset_test(Dataset):

    def __init__(self):
        self.x_test = x_test
        self.y_test = y_test
        self.num_samples = len(x_test)

    def __getitem__(self, index):
        return self.x_test[index], self.y_test[index]

    def __len__(self):
        return self.num_samples

data_train = CustomDataset_train()
data_test = CustomDataset_test()

batch_size= 1 #set batch size

train_dataloader = DataLoader(dataset = data_train, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(dataset = data_test, batch_size=batch_size, shuffle=True)

In [81]:
import torch.nn as nn
input_size = x.shape[1]
output_size = y.shape[1]

# define network architecture
class myModel(nn.Module):
    def __init__(self,input_dim, output_dim, hidden_size1 =128, hidden_size2 = 128):
        super(myModel, self).__init__()
        #define network layers
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, output_size)
      
    def forward(self,x):
        
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        
        out = self.fc3(out)
        return out
    
model = myModel(input_size, output_size)
print(model)

myModel(
  (fc1): Linear(in_features=1, out_features=128, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=128, out_features=1, bias=True)
)


In [82]:
#define loss function, optimizer and hyperparameters
loss_fn = torch.nn.MSELoss()
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  
total_step = len(train_dataloader)

In [83]:
def train(dataloader, model, loss_fn, optimizer, batch_size):
    model.train()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 1 == 0:
            loss, current = loss.item(), (batch+1) * batch_size
            print(f"Train_loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0
    model.eval()
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
    test_loss /= num_batches
    print(f"Avg_Test_loss: {test_loss:>8f} \n")

epochs = 1000
loss_train = []
loss_test = []
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer, batch_size)
    test(test_dataloader, model, loss_fn)


Epoch 1
-------------------------------
Train_loss: 18184.244141  [    1/   50]
Train_loss: 4137.812988  [    2/   50]
Train_loss: 28570.671875  [    3/   50]
Train_loss: 3213.902588  [    4/   50]
Train_loss: 13284.611328  [    5/   50]
Train_loss: 16283.027344  [    6/   50]
Train_loss: 7247.295898  [    7/   50]
Train_loss: 170245.765625  [    8/   50]
Train_loss: 1381.725586  [    9/   50]
Train_loss: 40181.472656  [   10/   50]
Train_loss: 2.066033  [   11/   50]
Train_loss: 32.806190  [   12/   50]
Train_loss: 1225.297974  [   13/   50]
Train_loss: 525.991821  [   14/   50]
Train_loss: 1092.697144  [   15/   50]
Train_loss: 158.274445  [   16/   50]
Train_loss: 455.806549  [   17/   50]
Train_loss: 59.094578  [   18/   50]
Train_loss: 386.634705  [   19/   50]
Train_loss: 1720.787476  [   20/   50]
Train_loss: 3585.836670  [   21/   50]
Train_loss: 1064.829956  [   22/   50]
Train_loss: 29.774652  [   23/   50]
Train_loss: 13589.519531  [   24/   50]
Train_loss: 18840.851562  [  

Train_loss: 1841.763794  [   18/   50]
Train_loss: 4343.867188  [   19/   50]
Train_loss: 374.725128  [   20/   50]
Train_loss: 289.792877  [   21/   50]
Train_loss: 5106.844238  [   22/   50]
Train_loss: 1126.231445  [   23/   50]
Train_loss: 13233.249023  [   24/   50]
Train_loss: 11424.575195  [   25/   50]
Train_loss: 2236.416992  [   26/   50]
Train_loss: 4531.441895  [   27/   50]
Train_loss: 152.393021  [   28/   50]
Train_loss: 5223.836914  [   29/   50]
Train_loss: 9742.352539  [   30/   50]
Train_loss: 44.084423  [   31/   50]
Train_loss: 971.525269  [   32/   50]
Train_loss: 120430.304688  [   33/   50]
Train_loss: 123.675163  [   34/   50]
Train_loss: 16753.781250  [   35/   50]
Train_loss: 8875.168945  [   36/   50]
Train_loss: 1990.450684  [   37/   50]
Train_loss: 3877.551514  [   38/   50]
Train_loss: 2249.694580  [   39/   50]
Train_loss: 2656.348145  [   40/   50]
Train_loss: 10413.843750  [   41/   50]
Train_loss: 15244.492188  [   42/   50]
Train_loss: 9791.545898  

Train_loss: 3966.278076  [   46/   50]
Train_loss: 154.847443  [   47/   50]
Train_loss: 1329.567993  [   48/   50]
Train_loss: 5851.888184  [   49/   50]
Train_loss: 537.559692  [   50/   50]
Avg_Test_loss: 9222.843429 

Epoch 10
-------------------------------
Train_loss: 11105.586914  [    1/   50]
Train_loss: 9212.641602  [    2/   50]
Train_loss: 13167.187500  [    3/   50]
Train_loss: 546.773071  [    4/   50]
Train_loss: 9495.874023  [    5/   50]
Train_loss: 668.692688  [    6/   50]
Train_loss: 574.466980  [    7/   50]
Train_loss: 7041.084473  [    8/   50]
Train_loss: 1441.819214  [    9/   50]
Train_loss: 820.537170  [   10/   50]
Train_loss: 1980.285278  [   11/   50]
Train_loss: 5992.939453  [   12/   50]
Train_loss: 1962.025879  [   13/   50]
Train_loss: 586.034668  [   14/   50]
Train_loss: 10333.160156  [   15/   50]
Train_loss: 9599.910156  [   16/   50]
Train_loss: 736.020813  [   17/   50]
Train_loss: 3060.677246  [   18/   50]
Train_loss: 2936.675781  [   19/   50]

Train_loss: 8025.860352  [   20/   50]
Train_loss: 1597.738525  [   21/   50]
Train_loss: 1084.655762  [   22/   50]
Train_loss: 15868.122070  [   23/   50]
Train_loss: 26124.671875  [   24/   50]
Train_loss: 15786.820312  [   25/   50]
Train_loss: 7689.314453  [   26/   50]
Train_loss: 11612.410156  [   27/   50]
Train_loss: 1844.108887  [   28/   50]
Train_loss: 878.354980  [   29/   50]
Train_loss: 1889.494385  [   30/   50]
Train_loss: 193.758484  [   31/   50]
Train_loss: 5774.352051  [   32/   50]
Train_loss: 4497.456543  [   33/   50]
Train_loss: 206.260681  [   34/   50]
Train_loss: 6828.341797  [   35/   50]
Train_loss: 2.904768  [   36/   50]
Train_loss: 219.322464  [   37/   50]
Train_loss: 346.511047  [   38/   50]
Train_loss: 1191.002441  [   39/   50]
Train_loss: 691.057434  [   40/   50]
Train_loss: 5.622520  [   41/   50]
Train_loss: 6416.840820  [   42/   50]
Train_loss: 939.683594  [   43/   50]
Train_loss: 4.191603  [   44/   50]
Train_loss: 2456.571533  [   45/   50

Train_loss: 4355.752441  [   42/   50]
Train_loss: 120367.382812  [   43/   50]
Train_loss: 21035.390625  [   44/   50]
Train_loss: 9165.726562  [   45/   50]
Train_loss: 1274.409668  [   46/   50]
Train_loss: 9261.847656  [   47/   50]
Train_loss: 101.198181  [   48/   50]
Train_loss: 51.870281  [   49/   50]
Train_loss: 903.672974  [   50/   50]
Avg_Test_loss: 10072.649662 

Epoch 19
-------------------------------
Train_loss: 7138.424316  [    1/   50]
Train_loss: 166.752121  [    2/   50]
Train_loss: 6848.409668  [    3/   50]
Train_loss: 419.441467  [    4/   50]
Train_loss: 55.309258  [    5/   50]
Train_loss: 1343.040527  [    6/   50]
Train_loss: 1330.448364  [    7/   50]
Train_loss: 920.257080  [    8/   50]
Train_loss: 9655.182617  [    9/   50]
Train_loss: 3816.885498  [   10/   50]
Train_loss: 5310.301758  [   11/   50]
Train_loss: 1892.503540  [   12/   50]
Train_loss: 884.085388  [   13/   50]
Train_loss: 7377.809082  [   14/   50]
Train_loss: 2651.882568  [   15/   50]


...

Train_loss: 1617.233032  [   24/   50]
Train_loss: 1684.420044  [   25/   50]
Train_loss: 578.514587  [   26/   50]
Train_loss: 242.502350  [   27/   50]
Train_loss: 114851.593750  [   28/   50]
Train_loss: 3357.849365  [   29/   50]
Train_loss: 485.101379  [   30/   50]
Train_loss: 970.170715  [   31/   50]
Train_loss: 11034.000977  [   32/   50]
Train_loss: 17278.464844  [   33/   50]
Train_loss: 41.324402  [   34/   50]
Train_loss: 28223.107422  [   35/   50]
Train_loss: 5590.889160  [   36/   50]
Train_loss: 6894.549805  [   37/   50]
Train_loss: 918.880981  [   38/   50]
Train_loss: 6588.600586  [   39/   50]
Train_loss: 87.307373  [   40/   50]
Train_loss: 17468.806641  [   41/   50]
Train_loss: 2006.472900  [   42/   50]
Train_loss: 2337.896484  [   43/   50]
Train_loss: 7650.276367  [   44/   50]
Train_loss: 1994.127808  [   45/   50]
Train_loss: 142.807709  [   46/   50]
Train_loss: 38.967762  [   47/   50]
Train_loss: 5469.327637  [   48/   50]
Train_loss: 2751.937012  [   49

Train_loss: 157.784393  [   35/   50]
Train_loss: 8477.955078  [   36/   50]
Train_loss: 5577.458008  [   37/   50]
Train_loss: 4127.065430  [   38/   50]
Train_loss: 33.656498  [   39/   50]
Train_loss: 1652.386597  [   40/   50]
Train_loss: 679.461487  [   41/   50]
Train_loss: 773.813354  [   42/   50]
Train_loss: 168.692673  [   43/   50]
Train_loss: 18849.689453  [   44/   50]
Train_loss: 29429.466797  [   45/   50]
Train_loss: 18.543802  [   46/   50]
Train_loss: 116106.031250  [   47/   50]
Train_loss: 4765.206055  [   48/   50]
Train_loss: 21864.849609  [   49/   50]
Train_loss: 4907.126953  [   50/   50]
Avg_Test_loss: 9880.689423 

Epoch 985
-------------------------------
Train_loss: 5412.146484  [    1/   50]
Train_loss: 5350.853516  [    2/   50]
Train_loss: 210.941544  [    3/   50]
Train_loss: 6715.196777  [    4/   50]
Train_loss: 19.958408  [    5/   50]
Train_loss: 1590.307739  [    6/   50]
Train_loss: 1050.424683  [    7/   50]
Train_loss: 111533.210938  [    8/   5

Train_loss: 23190.369141  [   47/   50]
Train_loss: 18807.894531  [   48/   50]
Train_loss: 1609.260742  [   49/   50]
Train_loss: 209.184921  [   50/   50]
Avg_Test_loss: 10021.765198 

Epoch 989
-------------------------------
Train_loss: 38.364506  [    1/   50]
Train_loss: 3342.417480  [    2/   50]
Train_loss: 56.098011  [    3/   50]
Train_loss: 71.021309  [    4/   50]
Train_loss: 5965.010742  [    5/   50]
Train_loss: 5286.324707  [    6/   50]
Train_loss: 159.265427  [    7/   50]
Train_loss: 6493.911133  [    8/   50]
Train_loss: 20845.943359  [    9/   50]
Train_loss: 5161.162598  [   10/   50]
Train_loss: 112964.937500  [   11/   50]
Train_loss: 2375.897461  [   12/   50]
Train_loss: 2795.752686  [   13/   50]
Train_loss: 5616.230469  [   14/   50]
Train_loss: 2.751308  [   15/   50]
Train_loss: 16151.335938  [   16/   50]
Train_loss: 885.877075  [   17/   50]
Train_loss: 84.556297  [   18/   50]
Train_loss: 2222.984619  [   19/   50]
Train_loss: 987.416687  [   20/   50]
T

Train_loss: 1435.583008  [   11/   50]
Train_loss: 3967.101562  [   12/   50]
Train_loss: 5521.685547  [   13/   50]
Train_loss: 154.323456  [   14/   50]
Train_loss: 12.066442  [   15/   50]
Train_loss: 769.918518  [   16/   50]
Train_loss: 6470.266113  [   17/   50]
Train_loss: 7492.947754  [   18/   50]
Train_loss: 170.026703  [   19/   50]
Train_loss: 2760.735596  [   20/   50]
Train_loss: 21105.169922  [   21/   50]
Train_loss: 1023.499573  [   22/   50]
Train_loss: 36.887421  [   23/   50]
Train_loss: 14.658922  [   24/   50]
Train_loss: 6858.813477  [   25/   50]
Train_loss: 134.257462  [   26/   50]
Train_loss: 1533.351074  [   27/   50]
Train_loss: 16548.056641  [   28/   50]
Train_loss: 5014.079590  [   29/   50]
Train_loss: 1152.171021  [   30/   50]
Train_loss: 111886.000000  [   31/   50]
Train_loss: 5748.803223  [   32/   50]
Train_loss: 1499.082520  [   33/   50]
Train_loss: 2271.909424  [   34/   50]
Train_loss: 5265.934082  [   35/   50]
Train_loss: 3436.136963  [   36

Train_loss: 960.535095  [   24/   50]
Train_loss: 1214.793091  [   25/   50]
Train_loss: 2.398727  [   26/   50]
Train_loss: 5640.621582  [   27/   50]
Train_loss: 161.618942  [   28/   50]
Train_loss: 27081.494141  [   29/   50]
Train_loss: 20362.775391  [   30/   50]
Train_loss: 4930.408203  [   31/   50]
Train_loss: 89.504723  [   32/   50]
Train_loss: 149.799438  [   33/   50]
Train_loss: 683.059875  [   34/   50]
Train_loss: 1324.028320  [   35/   50]
Train_loss: 28.176956  [   36/   50]
Train_loss: 1802.356567  [   37/   50]
Train_loss: 801.484924  [   38/   50]
Train_loss: 124.656219  [   39/   50]
Train_loss: 735.215271  [   40/   50]
Train_loss: 111915.484375  [   41/   50]
Train_loss: 3649.826660  [   42/   50]
Train_loss: 4407.822266  [   43/   50]
Train_loss: 6024.255859  [   44/   50]
Train_loss: 2306.726074  [   45/   50]
Train_loss: 10026.491211  [   46/   50]
Train_loss: 6358.946777  [   47/   50]
Train_loss: 6896.663086  [   48/   50]
Train_loss: 2749.341797  [   49/  